In [88]:
#Import and inspect data 
import pandas as pd
full_college_df = pd.read_csv('CollegeScorecard.csv')
crime_df = pd.read_csv('Crime_2015.csv')

/home/hussain/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (7,9,1427,1542,1561,1575,1725,1726,1727,1728) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [89]:
# Create dataframe college_df with only relevant columns 
cip = ['CIP11ASSOC', 'CIP11BACHL']
college_df = full_college_df.loc[:,['OPEID', 'INSTNM', 'CITY', 'STABBR', 'DISTANCEONLY', 'COSTT4_A', 'ADM_RATE_ALL'] + cip]

# Create a 'City and State' column 
crime_df['CityState'] = crime_df['City'] + ', ' + crime_df['State']
college_df['CityState'] = college_df['CITY'] + ', ' + college_df['STABBR']

# Merge college and crime data, keeping only values where both city and state are available
df = pd.merge(college_df, crime_df, on='CityState', how='inner')
df.describe()

,OPEID,DISTANCEONLY,COSTT4_A,ADM_RATE_ALL,CIP11ASSOC,CIP11BACHL,Murder,Rape,Robbery,AggravatedAssault,MotorVehicleTheft
count,3.307000e+03,3110.000000,1651.000000,1027.000000,3110.000000,3110.000000,3307.000000,3307.000000,3307.000000,3301.000000,3307.000000
mean,2.282641e+06,0.005466,26046.369473,0.694344,0.386817,0.387781,5.565195,41.561052,116.834654,264.039715,249.708134
std,4.981455e+06,0.073744,11545.327451,0.202725,0.669046,0.693288,3.119994,15.617338,64.846346,125.971276,138.892076
min,1.003000e+05,0.000000,6172.000000,0.048400,0.000000,0.000000,0.000000,3.200000,2.300000,25.500000,15.700000
25%,3.772000e+05,0.000000,18071.500000,0.562850,0.000000,0.000000,3.400000,30.900000,73.000000,179.400000,144.200000
50%,1.258406e+06,0.000000,24452.000000,0.724300,0.000000,0.000000,5.100000,37.400000,110.100000,240.800000,223.800000
75%,3.061500e+06,0.000000,31081.500000,0.838800,1.000000,1.000000,7.150000,49.500000,149.100000,327.100000,331.900000
max,8.209882e+07,1.000000,62425.000000,1.000000,2.000000,2.000000,19.300000,165.600000,351.900000,981.300000,718.600000


In [90]:
# Check for null values
print(df.isnull().any())

OPEID                False
INSTNM               False
CITY                 False
STABBR               False
DISTANCEONLY          True
COSTT4_A              True
ADM_RATE_ALL          True
CIP11ASSOC            True
CIP11BACHL            True
CityState            False
MSA                  False
ViolentCrime          True
Murder               False
Rape                 False
Robbery              False
AggravatedAssault     True
PropertyCrime         True
Burglary              True
Theft                 True
MotorVehicleTheft    False
State                False
City                 False
dtype: bool


In [91]:
#Create summary variable describing overall crime

#Slice columns related to crime
crime_cols = df.loc[:, "ViolentCrime":"MotorVehicleTheft"]
#Create new column that is the mean of all the crime variables
df['OverallCrime'] = crime_cols.mean(axis=1)
#Store OverallCrime as percentile data
df['OverallCrime'] = df['OverallCrime'].rank(pct=True)
#Remove other crime related variables
df = df.drop(list(crime_cols.columns), axis=1)
df.describe()

,OPEID,DISTANCEONLY,COSTT4_A,ADM_RATE_ALL,CIP11ASSOC,CIP11BACHL,OverallCrime
count,3.307000e+03,3110.000000,1651.000000,1027.000000,3110.000000,3110.000000,3307.000000
mean,2.282641e+06,0.005466,26046.369473,0.694344,0.386817,0.387781,0.500151
std,4.981455e+06,0.073744,11545.327451,0.202725,0.669046,0.693288,0.288704
min,1.003000e+05,0.000000,6172.000000,0.048400,0.000000,0.000000,0.000756
25%,3.772000e+05,0.000000,18071.500000,0.562850,0.000000,0.000000,0.250076
50%,1.258406e+06,0.000000,24452.000000,0.724300,0.000000,0.000000,0.500151
75%,3.061500e+06,0.000000,31081.500000,0.838800,1.000000,1.000000,0.741760
max,8.209882e+07,1.000000,62425.000000,1.000000,2.000000,2.000000,0.999698


In [92]:
#Remove rows that contain null values for non-crime variables
df = df.dropna(subset = ['DISTANCEONLY', 'COSTT4_A', 'ADM_RATE_ALL', 'CIP11ASSOC', 'CIP11BACHL'])
#Check to see if any null values remain
print(df.isnull().any())

OPEID           False
INSTNM          False
CITY            False
STABBR          False
DISTANCEONLY    False
COSTT4_A        False
ADM_RATE_ALL    False
CIP11ASSOC      False
CIP11BACHL      False
CityState       False
MSA             False
State           False
City            False
OverallCrime    False
dtype: bool
